In [1]:
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from IPython.display import display

import warnings

import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
precision = 3
values = [i*10**-precision for i in range(1,1001)]
e_max = 1.
e_values = [i*10**-precision for i in range(1,1000*int(e_max)+1)]
lambd = 0.0013581


# GUI components
n_slider = widgets.IntSlider(value=1000000, min=100, max=10000000, step=100, description='n')

a_slider = widgets.FloatSlider(
    value=0.10,
    min=1*10**-precision,
    max=1.0,
    step=0.001,
    description='alpha',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.'+str(precision)+'f',
)

b_slider = widgets.FloatSlider(
    value=0.050,
    min=1*10**-precision,
    max=1.0,
    step=0.001,
    description='beta',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.'+str(precision)+'f',
)

e_slider = widgets.FloatSlider(
    value=0.270,
    min=1*10**-precision,
    max=1.0,
    step=0.001,
    description='epsilon',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.'+str(precision)+'f',
)


# Layout
slider_box = widgets.VBox([n_slider, a_slider, b_slider, e_slider])

# FUNCTIONS

In [3]:
def calculateBeta(n, alpha, epsilon):
    c = (1.+np.exp(epsilon))/(np.exp(epsilon)-1.)
    beta = 2.*np.exp(-2*np.power(alpha/c,2)*float(n))
    return beta

In [4]:
%%latex
\begin{align}
 \beta={2e}^{-2{\lambda^2}n} = {2e}^{-2(\frac{\alpha}{\frac{1+e^\epsilon}{e^\epsilon-1}})^2 n} = {2e}^{-2(\frac{\alpha*e^\epsilon-1}{1+e^\epsilon})^2 n}
\end{align}

<IPython.core.display.Latex object>

In [5]:
def calculateLambdaFromNBeta(n, beta):
    lambd = np.sqrt(np.log(2/beta)/(2*n))
    return lambd

def calculateLambdaFromAlphaEpsilon(alpha, epsilon):
    lambd = alpha/(1+(np.exp(epsilon)))/(np.exp(epsilon)-1)
    return lambd

In [6]:
%%latex
\begin{align}
\lambda = \sqrt{\frac{\log{\frac{2}{\beta}}}{2n}}
\end{align}

<IPython.core.display.Latex object>

In [7]:
def calculateAlpha(n, beta, epsilon):
    lambd = calculateLambdaFromNBeta(n, beta)
    alpha = (1+(np.exp(epsilon)))/(np.exp(epsilon)-1)*lambd
    return alpha

In [8]:
%%latex
\begin{align}
\alpha = \frac{1+e^\epsilon}{e^\epsilon-1} \lambda =  \frac{1+e^\epsilon}{e^\epsilon-1} \sqrt{\frac{\log{\frac{2}{\beta}}}{2n}}
\end{align}

<IPython.core.display.Latex object>

In [9]:
def calculateEpsilon(n, alpha, beta):
    lambd = calculateLambdaFromNBeta(n, beta)
    epsilon = ((-alpha/lambd)-1)/(1-(alpha/lambd))
    return epsilon

In [10]:
%%latex
\begin{align}
\epsilon = \frac{\frac{-\alpha}{\lambda}-1}{1-\frac{\alpha}{\lambda}}
\end{align}

<IPython.core.display.Latex object>

In [11]:
def calculateN(alpha, beta, epsilon):
    top = np.power(1.+np.exp(epsilon), 2) * np.log(2/beta)
    bottom = 2*np.power(alpha, 2)*np.power((np.exp(epsilon)-1), 2)
    return top/bottom

In [12]:
%%latex
\begin{align}
n = \frac{(1 + e^\epsilon)^2 \log(2/\beta)}{2\alpha^2 (e^\epsilon - 1)^2}
\end{align}

<IPython.core.display.Latex object>

In [13]:
def updateAlpha(n, beta, epsilon):
    a = calculateAlpha(n, beta, epsilon)
    a = np.round(a, precision)
    
    if(a < 0):
        print "Warning, a < 0"
        
    if(a > 1):
        print "Warning, a > 1"
        a = 1.
    
    return a
    
def updateBeta(n, alpha, epsilon):
    b = calculateBeta(n, alpha, epsilon)
    b = np.round(b, precision)
    
    if(np.round(b, precision) <= 0):
        print "Warning, b < 0"
        b = 10**-precision
    if(b > 1):
        print "Warning, b > 1"
        b = 1.
    
    return b
     
def updateEpsilon(n, alpha, beta):
    e = calculateEpsilon(n, alpha, beta)
    e = np.round(e, precision)
    
    if(e < 0):
        print "Warning, e < 0"
        e = 10**-precision
    if(e > e_max):
        print "Warning, e > max"
        e = e_max
    
    return e

def updateN(alpha, beta, epsilon):
    n = calculateN(alpha, beta, epsilon)
    n = int(np.round(n))
    
    return n

def update(change): #(n, alpha, beta, epsilon):
    sliders = {
        'n': n_slider,
        'alpha': a_slider,
        'beta': b_slider,
        'epsilon': e_slider
    }
    
    # Pause notifications to prevent infinite loop
    for slider in sliders.values():
        slider.hold_trait_notifications()
    
    # Find which slider triggered the event
    changed = str(change['owner'].description)
    
    # Update the rest of the slider values
    unchanged = set(sliders.keys())-set(changed)

    equations = {
    'n': lambda n, alpha, beta, epsilon: updateN(alpha, beta, epsilon),
    'alpha': lambda n, alpha, beta, epsilon: updateAlpha(n, beta, epsilon),
    'beta': lambda n, alpha, beta, epsilon: updateBeta(n, alpha, epsilon),
    'epsilon': lambda n, alpha, beta, epsilon: updateEpsilon(n, alpha, beta) }
    
    for slider in unchanged:
        result = equations[slider](sliders['n'].value, sliders['alpha'].value, sliders['beta'].value, sliders['epsilon'].value)
        sliders[slider].value = result

In [14]:
n_slider.observe(update)
a_slider.observe(update)
b_slider.observe(update)
e_slider.observe(update)

# START HERE

In [15]:
display(slider_box)

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined

NameError: global name 'e_max' is not defined